In [1]:
from __future__ import print_function, division
%matplotlib nbagg
import timeit
import time
import os
from matplotlib import pyplot as plt

import tensorflow as tf
import numpy as np
from scipy import misc
import cv2
import xlsxwriter

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name="",
            op_dict=None,
            producer_op_list=None
        )
    return graph

class Text_segmentation(object):

    def __init__(self, path_to_pb, img_list, scale=0.3, mode='L'):
        self.graph = load_graph(path_to_pb)
        self.img_list = img_list
        self.scale = scale
        self.mode = mode

    def predictBaseline(self, print_result=True, gpu_device="0"):
        val_size = len(self.img_list)
        if val_size is None:
            print("No Inference Data available. Skip Inference.")
            return
        session_conf = tf.ConfigProto()
        session_conf.gpu_options.visible_device_list = gpu_device
        with tf.Session(graph=self.graph, config=session_conf) as sess:
            x = self.graph.get_tensor_by_name('inImg:0')
            predictor = self.graph.get_tensor_by_name('output:0')
            sess = tf.Session()


            print("Start segmenting---------------------------")
            timeSum = 0.0
            listImage = []
            for step in range(0, val_size):
                aTime = time.time()
                aImgPath = self.img_list[step]

                batch_x = self.load_img(aImgPath, 0.3, self.mode)

                # Run validation
                onlyFileName = os.path.basename(aImgPath)
                print(onlyFileName)
                aPred = sess.run(predictor,
                                       feed_dict={x: batch_x})
                curTime = (time.time() - aTime)*1000.0
                timeSum += curTime
                print("Prediction time: {:.2f} ms".format(curTime))
                if print_result:
                    misc.imsave('out'+ onlyFileName, aPred[0,:, :,0])
                    listImage.append('out' + onlyFileName)                            

            print("Stop segmenting---------------------------")

            return listImage


    def load_img(self, path, scale, mode):
        aImg = misc.imread(path, mode=mode)
        sImg = misc.imresize(aImg, scale, interp='bicubic')
        fImg = sImg
        if len(sImg.shape) == 2:
            fImg = np.expand_dims(fImg,2)
        fImg = np.expand_dims(fImg,0)
        return fImg
        
def read_image_list(pathToList):
    '''

    :param pathToList:
    :return:
    '''
    f = open(pathToList, 'r')
    filenames = []
    for line in f:
        if line[-1] == '\n':
            filenames.append(line[:-1])
        else:
            filenames.append(line)
    f.close()
    return filenames

def run(path_list_imgs="./imgs.lst", path_net_pb="./demo_nets/model100_ema.pb", report=False):
    
    workbook = xlsxwriter.Workbook('result.xlsx')
    worksheet = workbook.add_worksheet()
    worksheet.set_default_row(100)
    worksheet.set_column(0, 3, 100)

    listImage = read_image_list(path_list_imgs)
    textSegmentation = Text_segmentation(path_net_pb, listImage, mode='L')
    listPredictedImage = textSegmentation.predictBaseline()
    for index, imageName in enumerate(listPredictedImage):
        if report:
            worksheet.write(index,0, imageName)
            worksheet.insert_image(index,1, listImage[index])
        
        gray_test_image = cv2.imread(listImage[index],0)
        test_image = cv2.imread(listImage[index])        
        
        #-------------------find the baseline-----------------------------------------------------
        predicted_image = cv2.imread(imageName)
        predicted_image = misc.imresize(predicted_image, 10/3, interp='bicubic')
        gray_img = cv2.cvtColor(predicted_image, cv2.COLOR_BGR2GRAY)
        _,thresh = cv2.threshold(gray_img,30,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        baseline = []
        for cnt in contours:
            if cv2.contourArea(cnt) > 100: # to avoid too small component
                x,y,w,h = cv2.boundingRect(cnt)
                baseline.append((x,y,w,h))           
        baseline.sort(key=lambda x: x[1]) #sort by increase order (y-axis)
        #-------------------find the baseline-----------------------------------------------------
        
        result_box=[]        
        thresh = 1 - gray_test_image // 255
        for j,base in enumerate(baseline):
            x,y,w,h = base
            if j == 0:
                result_box.append((x,0,w,y+h)) 
            else:
                isFoundUpperLine = False
                
                #--------------------This block is needed for handwriting------???
                for jj in range(j-1,-1,-1):
                    previous_base = baseline[jj]
                    p_x, p_y, p_w, p_h = previous_base
                    
                    if (x in range(p_x, p_x + p_w)) or (p_x in range(x, x+w)):
                        result_box.append((x, p_y+p_h ,w, (y+h) - (p_y+p_h) ))
                        isFoundUpperLine = True
                        break
                #----------------------------------------------------------------
                if isFoundUpperLine == False:
                    for i in range(y,0,-1):
                        if np.sum(thresh[i,x:x+w],axis = 0) < 5:
                            result_box.append((x, i, w, y-i+h))
                            break
                        if i == 1:
                            result_box.append((x, i, w, y-i+h))
                            break
        for bbox in result_box:
            x,y,w,h = bbox
            cv2.rectangle(test_image,(x,y),(x+w,y+h),(255,0,0),2)
            cv2.imwrite("result_box"+imageName, test_image)
            if report:
                worksheet.insert_image(index,2, "result_box"+imageName)
    if report:    
        workbook.close()        
            
if __name__ == '__main__':
    run()

/home/phuongdinh1411/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.
Start segmenting---------------------------
106.png


/home/phuongdinh1411/.local/lib/python3.5/site-packages/ipykernel_launcher.py:79: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/phuongdinh1411/.local/lib/python3.5/site-packages/ipykernel_launcher.py:80: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Update time: 273.60 ms
107.png
Update time: 22.98 ms
108.png
Update time: 29.79 ms
109.png
Update time: 21.68 ms
110.png
Update time: 21.65 ms
111.png
Update time: 21.99 ms
112.png
Update time: 37.19 ms
113.png
Update time: 29.82 ms


/home/phuongdinh1411/.local/lib/python3.5/site-packages/ipykernel_launcher.py:70: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


114.png
Update time: 18.64 ms
115.png
Update time: 22.89 ms
116.png
Update time: 22.20 ms
117.png
Update time: 21.56 ms
118.png
Update time: 24.20 ms
119.png
Update time: 30.32 ms
120.png
Update time: 23.64 ms
121.png
Update time: 18.70 ms
122.png
Update time: 19.59 ms
123.png
Update time: 15.59 ms
124.png
Update time: 23.22 ms
125.png
Update time: 24.61 ms
126.png
Update time: 23.40 ms
127.png
Update time: 18.02 ms
128.png
Update time: 21.15 ms
129.png
Update time: 19.79 ms
130.png
Update time: 44.74 ms
131.png
Update time: 19.49 ms
132.png
Update time: 17.82 ms
133.png
Update time: 17.54 ms
134.png
Update time: 19.98 ms
135.png
Update time: 20.27 ms
136.png
Update time: 23.71 ms
137.png
Update time: 24.36 ms
138.png
Update time: 25.29 ms
139.png
Update time: 23.69 ms
140.png
Update time: 18.27 ms
Stop segmenting---------------------------


/home/phuongdinh1411/.local/lib/python3.5/site-packages/ipykernel_launcher.py:123: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
